In [1]:
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import string

In [2]:
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor
from slimit import minify

In [4]:
# connecting to my mongodb and get data out
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.movie_db
# create colletions
col = db['rotten_tomato']

In [23]:
# import data to dataframe
tomato_data = pd.DataFrame(list(col.find()))
goal_list = tomato_data.loc[tomato_data['critic(%)'] == 'na'][['name', 'url', 'year']]

In [24]:
goal_list

,name,url,year
0,Star Wars Ep. VII: The Force Awakens,https://www.rottentomatoes.com/m/star_wars_ep_...,2015
2,Black Panther,https://www.rottentomatoes.com/m/black_panther,2018
6,The Avengers,https://www.rottentomatoes.com/m/the_avengers,2012
7,Star Wars Ep. VIII: The Last Jedi,https://www.rottentomatoes.com/m/star_wars_ep_...,2017
11,Beauty and the Beast,https://www.rottentomatoes.com/m/beauty_and_th...,2017
13,Star Wars Ep. I: The Phantom Menace,https://www.rottentomatoes.com/m/star_wars_ep_...,1999
14,Star Wars Ep. IV: A New Hope,https://www.rottentomatoes.com/m/star_wars_ep_...,1977
20,Pirates of the Caribbean: Dead Manâs Chest,https://www.rottentomatoes.com/m/pirates_of_th...,2006
36,Star Wars Ep. III: Revenge of the Sith,https://www.rottentomatoes.com/m/star_wars_ep_...,2005
38,Beauty and the Beast,https://www.rottentomatoes.com/m/beauty_and_th...,1991


In [50]:
#search movie in rotten tomato
base_url = 'https://www.rottentomatoes.com/search/?search='
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=120)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        
        return r.text
    except:
        print(url + 'ERROR')
        pass

In [8]:
def get_movie_list(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    movie_list = {}
    try:
        movie_list = soup.find_all('script')[32]
    except:
        movie_list = 'na'
        
    return movie_list

In [10]:
def get_movie_url(data, movie, year):
    text = data.text
    parser = Parser()
    tree = parser.parse(text)
    # put the node data into a dict
    fields = []
    url_s = 'to find'
    for node in nodevisitor.visit(tree):
        if isinstance(node, ast.Assign):
            try:
                field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
                fields.append(field)   
            except:
                pass
    # check title and year
    #print(fields)
    for index, field in enumerate(fields):
        try:
            if(field['name'] == movie):
                year_dic = fields[index + 1]
                url = fields[index + 2]
                
                if year_dic['year'] == str(year):
                    url_s = url['url']
                
        except:
            pass
        
    return url_s

In [51]:
movie_find_result = []
for index, row in goal_list.iterrows():
    url_find_base = 'https://www.rottentomatoes.com'
    url_try = base_url + row['name']
    data = get_movie_list(url_try)
    url_find = get_movie_url(data, row['name'], row['year'])
    if url_find != 'to find':
        movie_find_result.append({
                'name': row['name'],
                'year': row['year'],
                'url': url_find_base + url_find
            })

In [52]:
movie_find_df = pd.DataFrame(
    data = movie_find_result,
    columns = ['name', 'year', 'url']
)
movie_find_df

,name,year,url
0,Black Panther,2018,https://www.rottentomatoes.com/m/black_panther...
1,Beauty and the Beast,2017,https://www.rottentomatoes.com/m/beauty_and_th...
2,Beauty and the Beast,1991,https://www.rottentomatoes.com/m/beauty_and_th...
3,The Passion of the Christ,2004,https://www.rottentomatoes.com/m/passion_of_th...
4,The Jungle Book,2016,https://www.rottentomatoes.com/m/the_jungle_bo...
5,Inside Out,2015,https://www.rottentomatoes.com/m/inside_out_2015
6,Spider-Man: Homecoming,2017,https://www.rottentomatoes.com/m/spider_man_ho...
7,Alice in Wonderland,2010,https://www.rottentomatoes.com/m/1221547-alice...
8,Suicide Squad,2016,https://www.rottentomatoes.com/m/suicide_squad...
9,Transformers,2007,https://www.rottentomatoes.com/m/transformers_...


In [45]:
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        return r.text
    except:
        print('ERROR')
        pass

def get_content(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    data = {}
    try:
        try:
            data['critic(%)'] = soup.find('span', class_='superPageFontColor').span.text
        except:
            data['critic(%)'] = 'na'
            
        try:
            data['critic_rate'] = soup.find('div', id='scoreStats').findChildren()[0].text[25:].strip()
        except:
            data['critic_rate'] = 'na'
            
        try:
            data['critic_count'] = soup.find('div', id='scoreStats').findChildren()[2].text[18:].strip()
        except:
            data['critic_count'] = 'na'
            
        try:
            data['critic_fresh'] = soup.find('div', id='scoreStats').findChildren()[7].text
        except:
            data['critic_fresh'] = 'na'
        
        try:
            data['critic_rot'] = soup.find('div', id='scoreStats').findChildren()[10].text
        except:
            data['critic_rot'] = 'na'
            
        try:
            data['user(%)'] = soup.find('div', class_='audience-score').find('span', class_='superPageFontColor').text[:2]
        except:
            data['user(%)'] = 'na'
            
        try:
            data['user_rate'] = soup.find('div', class_='audience-info').findChildren()[0].contents[2].strip()
        except:
            data['user_rate'] = 'na'
        
        try:
            data['user_number'] = soup.find('div', class_='audience-info').findChildren()[2].contents[2].strip()
        except:
            data['user_number'] = 'na'

    except:
        print('error')
    
    return data

In [67]:
get_content('https://www.rottentomatoes.com/m/sixth_sense')

{'critic(%)': '85',
 'critic_count': '151',
 'critic_fresh': '129',
 'critic_rate': '7.6/10',
 'critic_rot': '22',
 'user(%)': '90',
 'user_number': '1,036,091',
 'user_rate': '3.8/5'}

In [55]:
# get movie with corrected url
data_list = []
for index, row in movie_find_df.iterrows():
    data = get_content(row['url'])
    data_list.append(data)

In [59]:
movie_update_list = movie_find_df.join(pd.DataFrame(data=data_list, 
                                columns=['critic(%)', 'critic_rate', 'critic_count', 'critic_fresh', 'critic_rot', 'user(%)', 'user_rate', 'user_number']))

In [66]:
for index, row in movie_update_list.iterrows():
    col.find_one_and_update(
        {
            'name': str(row['name']),
            'year': int(row['year'])
        },
        {'$set':{
            'url': row['url'],
            'critic(%)': row['critic(%)'],
            'critic_rate': row['critic_rate'],
            'critic_count': row['critic_count'],
            'critic_fresh': row['critic_fresh'],
            'critic_rot': row['critic_rot'],
            'user(%)': row['user(%)'],
            'user_rate': row['user_rate'],
            'user_number': row['user_number']
        }}
    )

In [70]:
# get latest movie pending list and do second round search
tomato_data = pd.DataFrame(list(col.find()))
goal_list = tomato_data.loc[tomato_data['critic(%)'] == 'na'][['name', 'url', 'year']]
goal_list

,name,url,year
0,Star Wars Ep. VII: The Force Awakens,https://www.rottentomatoes.com/m/star_wars_ep_...,2015
6,The Avengers,https://www.rottentomatoes.com/m/the_avengers,2012
7,Star Wars Ep. VIII: The Last Jedi,https://www.rottentomatoes.com/m/star_wars_ep_...,2017
13,Star Wars Ep. I: The Phantom Menace,https://www.rottentomatoes.com/m/star_wars_ep_...,1999
14,Star Wars Ep. IV: A New Hope,https://www.rottentomatoes.com/m/star_wars_ep_...,1977
20,Pirates of the Caribbean: Dead Manâs Chest,https://www.rottentomatoes.com/m/pirates_of_th...,2006
36,Star Wars Ep. III: Revenge of the Sith,https://www.rottentomatoes.com/m/star_wars_ep_...,2005
59,It,https://www.rottentomatoes.com/m/it,2017
70,Star Wars Ep. II: Attack of the Clones,https://www.rottentomatoes.com/m/star_wars_ep_...,2002
72,Star Wars Ep. VI: Return of the Jedi,https://www.rottentomatoes.com/m/star_wars_ep_...,1983
